# No answer tool 
*Prototype for no_answer_tool*

here we demonstrate how the tool could work with all processes and a first output

## Step 1 search for the question(s) and retrieve answers

*sub steps:*   
- get keywords   
- generate automatic experiences   
- get people_also_ask questions   
- get answers name and link from google first page   
- scrape the text from each link in the answer   



In [10]:
# TODO ask user for input

Q0 = "damselfish water"

In [11]:
# check if it is a question and etxtract main tokens
Q0_type = 1 if len(Q0.split(" "))>2 or "?" in Q0 else 0


import re
def get_main_tokens(input):
    # clean input
    clean_input = re.sub( "[^A-Za-z0-9\s]", "", input).lower()
    Q0_tokens= clean_input.split(" ")  # TODO:  this can be improved with tokenizer
    lst2 = sorted(Q0_tokens, key=len, reverse=True) 
    return lst2[:2]

if Q0_type == 1:
    tokens = get_main_tokens(Q0)
else:
    tokens = Q0.split(" ")
print(tokens)

['damselfish', 'water']


In [12]:
# generate questions based on tokens
# this can be improved with some simple ML system 
def generate_basic_qs(tokens):
    q_start = ["Who is ", "where is ", "what is ", "when is ", "which is ", "How is ", "Why is " ]
    q_start = ["Who ", "where  ", "what  ", "when  ", "which  ", "How ", "Why " ]

    q_complete = [ a + " ".join(tokens)+" ?" for a in q_start]
    return q_complete
    
basic_q = generate_basic_qs(tokens)
if Q0_type == 1:
    basic_q.append(Q0)
print(basic_q)


['Who damselfish water ?', 'where  damselfish water ?', 'what  damselfish water ?', 'when  damselfish water ?', 'which  damselfish water ?', 'How damselfish water ?', 'Why damselfish water ?']


In [13]:
# get related questions (and select the most useful)
# Q: should we eliminate questions that do not have related question or are those the most interesting?
import people_also_ask

def get_advanced_q(basic_q, limit=10):
    """gets questions through people also ask package
    returns list of quetsion of length ~= limit"""
    advanced_q = []
   
    for n, q in enumerate(basic_q):
        print(f"\n getting related questions for question {n+1}: \n    {q}..")
        g_q = people_also_ask.get_related_questions(q, 5)
        print("..done!")
        sel_n_q = [n_q for n_q in g_q if n_q not in advanced_q]
        print("\n".join(sel_n_q))
        if sel_n_q:    
            advanced_q = advanced_q + sel_n_q + [q]
        print(f"got {len(advanced_q)} questions. {limit-len(advanced_q)} still to go\n")       
    
    return advanced_q, basic_q
    
    
a_q, b_q = get_advanced_q(basic_q)
print( "basic_q: ", b_q)
print("advanced q:", a_q)


 getting related questions for question 1: 
    Who damselfish water ?..
..done!
How many damselfish are in a tank?
Can damselfish live with clownfish?
What fish can I put with my clownfish?
How do you catch a damselfish in an aquarium?
Can you keep different species of clownfish together?
Can damselfish live in freshwater?
got 7 questions. 3 still to go


 getting related questions for question 2: 
    where  damselfish water ?..
..done!
What do damselfish eat in the ocean?
got 9 questions. 1 still to go


 getting related questions for question 3: 
    what  damselfish water ?..
..done!

got 9 questions. 1 still to go


 getting related questions for question 4: 
    when  damselfish water ?..
..done!
How many damselfish are in one tank?
How often do I feed damsels?
What is the habitat of the damselfish?
Will damsels kill each other?
got 14 questions. -4 still to go


 getting related questions for question 5: 
    which  damselfish water ?..
..done!
How can you tell if a damselfish

In [14]:
# TODO: ADD here way to filter questionŝ and get only the most relevant (10 q) 

In [15]:
# get first page of google results

from googleapi import google

def get_first_page_results(q, num_page = 1, limit = 20):
    search_results = None
    n=0
    while not search_results and n<=limit:
        n+=1
        print(f"getting first page results: try {n} of {limit}...")
        search_results = google.search(q, num_page)
    print("...done!")
    return search_results

advanced_q = a_q

answer_list = []
for q in advanced_q:
    print(f"\n the question is : {q}")
    search_res = get_first_page_results(q)
    answer_dic={}
    answer_dic['question'] = q
    answer_dic['link'] = [a.link for a in search_res]
    answer_dic['name'] = [a.name for a in search_res]
    answer_list.append(answer_dic)
print(answer_list)

# TODO: ADD second cycle to get all results


 the question is : How many damselfish are in a tank?
getting first page results: try 1 of 20...
...done!

 the question is : Can damselfish live with clownfish?
getting first page results: try 1 of 20...
getting first page results: try 2 of 20...
getting first page results: try 3 of 20...
getting first page results: try 4 of 20...
getting first page results: try 5 of 20...
...done!

 the question is : What fish can I put with my clownfish?
getting first page results: try 1 of 20...
getting first page results: try 2 of 20...
getting first page results: try 3 of 20...
getting first page results: try 4 of 20...
getting first page results: try 5 of 20...
getting first page results: try 6 of 20...
getting first page results: try 7 of 20...
getting first page results: try 8 of 20...
getting first page results: try 9 of 20...
getting first page results: try 10 of 20...
getting first page results: try 11 of 20...
getting first page results: try 12 of 20...
getting first page results: try 13 

In [16]:
for my_dic in answer_list:
    print("\n \n Q:", my_dic['question'] )
    for n,i in enumerate(range(len(my_dic['name']))):
        print(my_dic["name"][n])
        print(my_dic["link"][n], "\n")        


 
 Q: How many damselfish are in a tank?
Damselfish: Complete Care Guide for Blue, Yellow and More ...www.fishkeepingworld.com › dam...
https://www.fishkeepingworld.com/damselfish/ 

Damselfish Care | Tropical Fish Hobbyist Magazinewww.tfhmagazine.com › saltwater
https://www.tfhmagazine.com/articles/saltwater/damsels-in-my-aquarium 

Damselfish: 10 Types & How to Keep Them - Build Your ...www.buildyouraquarium.com › da...
https://www.buildyouraquarium.com/damselfish/ 

Multiple Damsel fish in one tank? - Beginners Discussion ...www.nano-reef.com › topic › 299...
https://www.nano-reef.com/forums/topic/299386-multiple-damsel-fish-in-one-tank/ 

Damselfish - Aquarium store depotaquariumstoredepot.com › news
https://aquariumstoredepot.com/blogs/news/116712964-is-there-a-sane-damselfish-yes-they-exist 

multiple damselfish ? | Ultimate Reefwww.ultimatereef.net › threads
https://www.ultimatereef.net/threads/multiple-damselfish.319713/ 

Damselfish - Wikipediaen.wikipedia.org › wiki › Damsel

In [17]:
# scrape the answer page
import requests
from bs4 import BeautifulSoup 

def scrape_answer(answer_link, mode = "text"):
    """scrapes the given link and return text opr other parts of the html"""
    resp = requests.get(answer_link)
    soup = BeautifulSoup(resp.text, 'html.parser')

    answer_text = []
    for i in soup.find_all("p"):
        
        if mode == "text":
            ans = re.sub("[^\S][\W]", "", i.text)
            if len( ans.split(" ")) > 2:
                answer_text.append(ans)
    return answer_text

q = answer_list[0]['question']
print(q)
answer_link = answer_list[0]['link'][0]
print(answer_link)

scrape_answer(answer_link)[:5]

How many damselfish are in a tank?
https://www.fishkeepingworld.com/damselfish/


['The Damselfish belong to the Pomacentridae family, of which there is over 250 species.',
 'Whilst the majority of these Damselfish species are found in saltwater, certain species can survive in brackish waters.',
 'The Damselfish are renowned for their bright colors, hardiness and ease to keep.',
 'xTo view this video please enable JavaScript, and consider upgrading to a web browser that supports HTML5 video\nNEON_TETRA video It’s important to mention that the Damselfish is an extremely large group of fish and as a result each species of Damsel has their own quirks and individuality.',
 ' To view this video please enable JavaScript, and consider upgrading to a web browser that supports HTML5 video\n']


## Metrics

Below here there are the functions to calculate metrics for each answer page. The following metrics are envisaged:

- Web ranking:
    - Domain authority
    - Page authority

- Quality of page:
    - Difficulty of text (grade)
    - Type /number of media in page
    - Length of text

- Quality of response:
    - Sentence similarity
    - Salience of kw in text
    
All funcitons to calculate metrics should have question and (answer) link as arguments, additional arguments only as kwargs 

In [ ]:
q = answer_list[0]['question']
answer_link = answer_list[0]['link'][0]

In [ ]:
# Web metrics

def get_domain_authority

In [ ]:
# get most similar sentence

# compute similarity

# get other info from the website

# structure output